今天的计算机是高度并行的系统，由多个CPU核、多个GPU、多个处理单元组成。通常每个CPU核有多个线程，每个设备通常有多个GPU，每个GPU有多个处理单元。总之，我们可以同时处理许多不同的事情，并且通常是在不同的设备上。不幸的是，Python并不善于编写并行和异步代码，至少在没有额外帮助的情况下不是好选择。归根结底，Python是单线程的，将来也是不太可能改变的。

因此在诸多的深度学习框架中，MXNet和TensorFlow之类则采用了一种异步编程（asynchronous programming）模型来提高性能，而PyTorch则使用了Python自己的调度器来实现不同的性能权衡。对PyTorch来说GPU操作在默认情况下是异步的。当调用一个使用GPU的函数时，操作会排队到特定的设备上，但不一定要等到以后才执行。这允许我们并行执行更多的计算，包括在CPU或其他GPU上的操作。

因此，了解异步编程是如何工作的，通过主动地减少计算需求和相互依赖，有助于我们开发更高效的程序。这能够减少内存开销并提高处理器利用率。

In [4]:
import os
import subprocess
import numpy
import torch
from torch import nn
from d2l import torch as d2l

作为热身，考虑一个简单问题：生成一个随机矩阵并将其相乘。让我们在NumPy和PyTorch张量中都这样做，看看它们的区别。请注意，PyTorch的tensor是在GPU上定义的。

In [5]:
# GPU计算热身
device = d2l.try_gpu()
a = torch.randn(size=(1000, 1000), device=device)
b = torch.mm(a, a)

with d2l.Benchmark('numpy'):
    for _ in range(10):
        a = numpy.random.normal(size=(1000, 1000))
        b = numpy.dot(a, a)

with d2l.Benchmark('torch'):
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)

numpy: 0.4240 sec
torch: 0.0010 sec


默认情况下，GPU操作在PyTorch中是异步的。强制PyTorch在返回之前完成所有计算，这种强制说明了之前发生的情况：计算是由后端执行，而前端将控制权返回给了Python

异步计算指的是在发起一个计算任务后，不必等待这个任务完成，就可以继续执行后续代码。任务会在后台（后端）运行，直到最终完成，其结果可以在需要时再获取。这种方式能提升资源利用率和整体执行速度，尤其在处理 I/O 或 GPU 等并行设备时效果明显。

在这个例子中，PyTorch 的 GPU 操作默认是异步的。当你调用诸如 torch.mm(a, a) 等函数时，这些操作会被提交到 GPU 的队列中，由后端（即 GPU 驱动或计算库）执行，而 Python 前端立即返回，将控制权还给 Python 解释器。这就意味着 CPU 不必等待 GPU 计算任务结束，而可以继续执行其他代码。当你强制同步或等待结果时，才会真正阻塞，直到 GPU 完成所有操作。

这种设计让多个 GPU 操作可以并行进行（即便在 Python 单线程环境下），从而提高整体性能。

In [6]:
with d2l.Benchmark():
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)
    torch.cuda.synchronize(device)

Done: 0.0733 sec


广义上说，PyTorch有一个用于与用户直接交互的前端（例如通过Python），还有一个由系统用来执行计算的后端。

用户可以用各种前端语言编写PyTorch程序，如Python和C++。不管使用的前端编程语言是什么，PyTorch程序的执行主要发生在C++实现的后端。由前端语言发出的操作被传递到后端执行。后端管理自己的线程，这些线程不断收集和执行排队的任务。请注意，要使其工作，后端必须能够跟踪计算图中各个步骤之间的依赖关系。因此，不可能并行化相互依赖的操作

In [7]:
x = torch.ones((1, 2), device=device)
y = torch.ones((1, 2), device=device)
z = x * y + 2
z

tensor([[3., 3.]], device='cuda:0')

每当Python前端线程执行前三条语句中的一条语句时，它只是将任务返回到后端队列。当最后一个语句的结果需要被打印出来时，Python前端线程将等待C++后端线程完成变量z的结果计算。后端线程式异步处理，从而提高性能。这种设计的一个好处是Python前端线程不需要执行实际的计算。因此，不管Python的性能如何，对程序的整体性能几乎没有影响。

这里虽然仍然是命令式编程，但是通过异步机制实现了更高效的并行计算

深度学习框架可以将Python前端的控制与后端的执行解耦，使得命令可以快速地异步插入后端、并行执行。

异步产生了一个相当灵活的前端，但请注意：过度填充任务队列可能会导致内存消耗过多。建议对每个小批量进行同步，以保持前端和后端大致同步。

芯片供应商提供了复杂的性能分析工具，以获得对深度学习效率更精确的洞察。

“解耦”指的是将系统中彼此关联紧密的部分分开，使它们各自独立运行，减少相互之间的依赖，从而更容易维护、扩展和优化。

在这里，“前端控制和后端执行的解耦”具体指：

- 前端控制：
Python代码（前端）负责构造和发出计算任务。例如，调用诸如 torch.mm(a, a) 这样的函数，将操作提交到后端。

- 后端执行：
后端（通常由C++实现的底层库和GPU驱动）接收到前端提交的任务后，在自己的线程中排队并异步执行这些计算操作。

二者解耦的好处在于，前端只负责命令的发送，不需要等待实际的计算完成，从而能快速响应和发出更多任务；而后端则专注于高效执行这些任务。这样设计既提高了并行计算效率，也使整个系统更为灵活和高效。